In [5]:
# standard ds imports
import numpy as np
import pandas as pd

# for wrangling and exploration
import wrangle
# import explore
import model

# for visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# filter out noise
import warnings
warnings.filterwarnings('ignore')

# align markdown tables to the left

In [6]:
%%html
<style>
table {float:left}
</style>

# MYERS BRIGGS TWITTER ASSESSMENT

This project is designed to analyze the relationship between the 16 Myers Briggs personality types and twitter behavior. The results.....

----

## Executive Summary:

Goals:
- analyze relationships between tweets and the myers briggs 16 personalities
- build a classification model to predict a personality type based on a user's tweets

Key Findings:
- stuff
- things

Takeaways:
- Our best model, a multinomial Naive Bayes using TF-IDF, performed at 17.5% accuracy on the test dataset. This barely outperforms the baseline at 17.2%.
- things

Recommendations:
- Due to the low accuracy, it is not recommended to use this for individual predictions.
- However, looking at how types and domains interact can provide business insights for targeted marketing and training for known personality types

----
## 1. Planning
 - Create deliverables:
     - README
     - final_report.ipynb
     - working_report.ipynb
     - canva presentation
 - Bring over functional wrangle.py, explore.py, and model.py files
 - Acquire the data from the Kaggle and Twitter via acquire.py's web scraping. Save the data locally.
 - Prepare and split the data via the prepare.py functions
 - Explore the data and define hypothesis. Run the appropriate statistical tests in order to accept or reject each null hypothesis. Document findings and takeaways.
 - Model a baseline in predicting personality type and document the accuracy.
 - Fit and train classification models to predict personality type or domain on the train dataset.
 - Evaluate the models by comparing the train and validation data.
 - Select the best model and evaluate it on the train data.
 - Develop and document all findings, takeaways, recommendations and next steps.

----
## 2. Data Wrangling
This step calls the wrangle function from the wrangle.py. This function:
- reads the fivezerominpull.plk file saved locally

#### Data Dictionary

| Target | Type | Description |
| ---- | ---- | ---- |
| type | str | the 16 myers briggs personality types |

| Feature Name | Type | Description |
| ---- | ---- | ---- |
| name | str | the owner of the twitter account scraped |
| lemmatized | str | the lemmatized version of past 100 tweets scraped by user |
| sentiment | float | the compound sentiment score |
| word_count | int | the count of words in the combined 100 tweets after lemmatization |
| message_length | int | the count of characters in the combined 100 tweets after lemmatization |
| i_e | int | 0 for introvert, 1 for extrovert |
| n_s | int | 0 for intuitive, 1 for sensing |
| t_f | int | 0 for thinking, 1 for feeling |
| j_p | int | 0 for judging, 1 for perceiving |
| personality_domain | str | which of the 4 domains the personality type is in |
| freq | dict | dictionary of count of each unique word |

In [3]:
# call the wrangle function from wrangle.py
df = wrangle.wrangle()

In [4]:
df.head()

,type,name,lemmatized
0,infp,alicia keys,dragon ball zone beerus less december back jun...
1,infp,farhan akhtar,lift spirits theme song cinemas nov n gurmmeet...
2,infp,calum hood,song video journey proud guys finally hope yal...
3,infp,john green,lee html great tell thx lee html rs molloy ahh...
4,infp,noel schajris,puebla magi buenos dias va sabado ustedes noch...


In [5]:
# split

In [6]:
# verify the split

### Wrangling Takeaways
- stuff
- things

----
## 2. Data Exploration

What does the data look like? (Percent population -> flow chart with ratios)

### Question 1: What scale of groupings shows a significant relationship in sentiment? Pairs, types, or domains?

In [ ]:
# where is there a significant difference in samples?

#### Hypothesis 1:

    Ho ->
    Ha ->
    T-Test, One-Sample, Two-Tailed

### Question 1 Takeaways
- stuff
- things

---
### Question 2a: What words are seen across all personality types?

#### Hypothesis 2:

    Ho ->
    Ha ->
    T-Test, One-Sample, Two-Tailed

### Question 2b: What words are unique to introvert or extrovert?

### Question 2 Takeaways
- stuff
- things

---
### Question 3: Is there a relationship between word count and personality type?

#### Hypothesis 3:

    Ho ->
    Ha ->
    T-Test, One-Sample, Two-Tailed

### Question 3 Takeaways
- stuff
- things

### Exploration Takeaways
- stuff
- things
- more stuff

---
## Data Modeling
The goal is to maximize accuracy.

| Features Dropped | Features Kept |
| ---- | ---- |
| pairs | lemmatized |
| domain | bow |
| sentiment | TF-IDF |
| message_length |  |
| word_count |  |
|  |  |

In [ ]:
# get the baseline accuracy
model.get_baseline(df)

In [1]:
# get the model results on train and validate
model.top_3_16_personalities(df)

In [ ]:
# get the best model's results on test
model.test_16_personalities(df)

### Modeling Takeaways:
- The best model was the TF-IDF Multinomial Naive Bayes model at % accuracy on test.
- The other models showed strong signs of overfitting the data on train, but were producing similar results on validate.

---
## Conclusion
Main conclusion note here

### Recommendations
- stuff
- things

### Next Steps:
- stuff
- things
---